In [5]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import matplotlib.pyplot as plt
import math 
import numpy as np

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.


In [8]:
#'W' and 'V' operators.
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

W_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]

V_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]

#A random selection mechanism
#To do this, we can make it randomly give us a state of 4 qubits, and then just disregard one.
def generate_random():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    qc.x(1)
    qc.h(0)
    qc.measure_all()

    backend = BasicSimulator()
    compiled = transpile(qc, backend)
    job_sim = backend.run(compiled, shots=1) #we just want to measure this once to give us one result
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    result = list(counts.keys())[0]

    if result == "00":
        return 1
    if result == "01":
        return 2
    if result == "10":
        return 3
    if result == "11":
        return generate_random()

alice_operators = []
bob_operators = []

alice_results = []
bob_results = []

N = 100

iterations = int((9*N)/2)

for i in range(iterations):
    
    #make an entangled pair in the right state
    q = QuantumCircuit(2, 2) 
    q.x(1)       
    q.h(0)       
    q.cx(0,1)   
    q.z(0)       
    
    #Alice will randomly select an operator
    alice_operator = generate_random()
    alice_operators.append(alice_operator)

    #Bob will randomly select an operator
    bob_operator = generate_random()
    bob_operators.append(bob_operator)
    
    #Alice measures operator Ai on her qubit
    if alice_operator == 1:
        q.h(0)
    if alice_operator == 2:
        q.unitary(W_transform_matrix, [0])
    if alice_operator == 3:
        pass

    q.measure(0,0)

    #Bob measures operator Bj on is qubit
    if bob_operator == 1:
        q.unitary(W_transform_matrix, [1])
    if bob_operator == 2:
        pass
    if bob_operator == 3:
        q.unitary(V_transform_matrix, [1])

    q.measure(1,1)

    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1) 
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    
    result = list(counts.keys())[0]
    alice_results.append(int(result[0])) 
    bob_results.append(int(result[1]))

#now, let's make the key

alice_key = []
bob_key = []

#alice operator guide --> 1 = X, 2 = W, 3 = Z
#bob operator guide --> 1 = W, 2 = Z, 3 = V

xw = []
xv = []
zw = []
zv = []

def convert(num):
    if num == 0:
        return 1
    if num == 1:
        return -1
        
for i in range(0, iterations):
    #this block checks if they used the same operators
    if (alice_operators[i] == 2 and bob_operators[i] == 1) or (alice_operators[i] == 3 and bob_operators[i] == 2):
        alice_key.append(alice_results[i])
        bob_key.append(1 - bob_results[i])  # Bob inverts his bit

    #if they didn't we'd convert the bits to -1 and +1
    a = convert(alice_results[i])
    b = convert(bob_results[i])
        
    if alice_operators[i] == 1 and bob_operators[i] == 1: #XW
        xw.append(a*b)

    if alice_operators[i] == 1 and bob_operators[i] == 3: #XV
        xv.append(a*b)

    if alice_operators[i] == 3 and bob_operators[i] == 1: #ZW
        zw.append(a*b)

    if alice_operators[i] == 3 and bob_operators[i] == 3: #ZV
        zv.append(a*b)
        
xw = sum(xw)/len(xw) if xw else 0
xv = sum(xv)/len(xv) if xv else 0
zw = sum(zw)/len(zw) if zw else 0
zv = sum(zv)/len(zv) if zv else 0

s = abs(xw-xv+zw+zv)

if s< 2.2: #not using 2 root 2 here as we want to allow values that are close to 2 root 2. 
    print("Attacker detected!")

if alice_key == bob_key:
    print("Shared key: ", alice_key)

else:
    print("Alice's key: ", alice_key)
    print("Bob's key: ", bob_key)
    
print("S = ", s)

Shared key:  [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
S =  2.728756913784701
